In [ ]:
#!pip install geemap
# !pip install pycrs
# !pip install pyshp
# !pip install oeel

In [2]:
from datetime import datetime 
#from google.colab import drive
import os
import ee
import geemap
import pycrs
geemap.ee_initialize()
ee.Authenticate()


Successfully saved authorization token.


In [3]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
# Load Soil Moistures Sites (SMS)
SM_Sites = geemap.shp_to_ee('../02_Data/03_Vector_data/Station_coordinates.shp')
# Making a poligon over the extention of the Soil Moistures Sites
ROI = SM_Sites.geometry().bounds().buffer(1000) 
area_ROI = ROI.area()

# Center Region of Interst
Map.centerObject(SM_Sites,12)
Map.setOptions("SATELLITE")
Map.addLayer(ROI,{},'ROI')
vis_params = {
    'color': 'red',
    'colorOpacity': 1,
    'pointSize': 1,
    'pointShape': 'circle',
    'width': 1,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
Map.addLayer(SM_Sites, vis_params, 'Sites')

Map

Map(bottom=417.0, center=[46.708955316924424, 10.693252538661627], controls=(ZoomControl(options=['position', …

$J_{k}(x,y) =\frac{I_{k}}{N} \sum_{i=1}^{N} \frac{I_{i}(x,y))}{I_{i}} , 1\leq k\leq N.$
Where:
  - J is the corrected intensity at the pixel location (x, y) for image k
  - I original pixel value
  - i=
  

In [199]:
# Multitemporal Despeckle function 
def multitemporalDespeckle(images, radius, units, opt_timeWindow=None):
        """
        Function for multi-temporal despeckling
            - images: S1 Image Collection
            - radius: radius of the square Kernel
            - units: unit of radius the Kernel
            - opt_timeWindow: Temporal time window
        
        Note: this is a simplified function of the original (Quegan, 2000),where the correlation 
        is considered null.
        In this sense, this filter must be applied only in areas with temporal changes, not bare-soil and cities. 
        """
        #bandNames = ee.Image(images.first()).bandNames()
        #NewbandNames = bandNames.map(lambda b: ee.String(b).cat('_tmpfilter'))
        #bandNamesMean = bandNames.map(lambda b: ee.String(b).cat('_mean'))
        #bandNamesRatio = bandNames.map(lambda b: ee.String(b).cat('_ratio'))
        
        def mapMeanSpace(i):
            '''
            
            '''
            pol_name = ee.String(i.bandNames().get(0)) # polarization band name
            spatial_filter_name = pol_name.cat('_multilook')
            ## bandNamesMean_pol = pol_name.map(lambda b: ee.String(b).cat('_mean'))
            ratio_name = pol_name.cat('_ratio')
            
            reducer = ee.Reducer.mean()
            kernel = ee.Kernel.square(radius, units)
            mean = i.reduceNeighborhood(reducer, kernel).rename(spatial_filter_name) # multilook (average)
            ratio = i.divide(mean).rename(ratio_name) # Intensity/ Imean
            return (i.addBands(mean).addBands(ratio))

        if opt_timeWindow == None:
            timeWindow = dict(before=-3, after=3, units='month')
        else:
            timeWindow = opt_timeWindow
        
        # compute spatial average (kernel multilook) for all images
        
        meanSpaceVV = images.select("VV").map(mapMeanSpace) # Spatial filter in VV
        meanSpaceVH= images.select("VH").map(mapMeanSpace) # Spatial filter in VH
        meanSpace = meanSpaceVV.combine(meanSpaceVH)
        
        # computes a multi-temporal despeckle function for a single image

        def multitemporalDespeckleSingle(image):
            t = image.date() # get image date
            fro = t.advance(ee.Number(timeWindow['before']), timeWindow['units']) # temporal window end 
            to = t.advance(ee.Number(timeWindow['after']), timeWindow['units']) # temporal window start
            
            pol_band_name = ee.String(image.bandNames().get(0))
            #print(pol_band_name.getInfo()=="VV")
            #print((ee.String(image.bandNames().get(0)).getInfo())=='VV')
            #print((ee.String(image.bandNames().get(0)).getInfo())==ee.String('VV'))
            #print(ee.String(image.bandNames().get(0)))
            #print(image.select("VV.*").bandNames().size().getInfo()) 
            threshold = ee.String(ee.Number(timeWindow['after']))
            #print(ee.String(image.bandNames().get(0)).getInfo())
            
            # .cat(ee.String(ee.Number(threshold)))
            NewbandNames = pol_band_name.cat("_").cat(ee.String(ee.Number(threshold)))\
                .cat("_").cat(ee.String((timeWindow['units']))).cat("_temporalfilter")
            
            # MeanSpace2 is an Image Collection of all the Ratios within the time window 
            #mean_space_pol = meanSpace.select(ee.String(image.bandNames().get(0)).cat(".*"))
            #mean_space_pol = meanSpace.select("VV.*")
            meanSpace2_VV = ee.ImageCollection(meanSpaceVV.select(".._ratio")).filterDate(fro, to) \
                .filter(ee.Filter.eq('relativeOrbitNumber_start', image.get('relativeOrbitNumber_start')))
            Ik_VV = image.select(".._multilook") # Ik
            Img_temporal_filtered_VV = Ik_VV.divide(meanSpace2_VV.count()).multiply(meanSpace2_VV.sum())\
                    .rename(NewbandNames)
            
            meanSpace2_VH = ee.ImageCollection(meanSpaceVH.select(".._ratio")).filterDate(fro, to) \
                .filter(ee.Filter.eq('relativeOrbitNumber_start', image.get('relativeOrbitNumber_start')))
            
            Ik_VH = image.select(".._multilook") # Ik
            Img_temporal_filtered_VH = Ik_VH.divide(meanSpace2_VH.count()).multiply(meanSpace2_VH.sum())\
                    .rename(NewbandNames)
            
            return image.addBands(Img_temporal_filtered_VV).addBands(Img_temporal_filtered_VH)\
                .set('system:time_start',image.get('system:time_start'))
            #return (Ik)
        # .select("VV.*") # Select the VV polarization bands
        
        Temp_filter_VV = meanSpace.select("VV.*").map(multitemporalDespeckleSingle) # Temporal filter in VV
        Temp_filter_VH= meanSpace.select("VH.*").map(multitemporalDespeckleSingle) # Temporal filter in VH
        meanSpace2 = Temp_filter_VV.combine(Temp_filter_VH)
        
        #return meanSpace.map(multitemporalDespeckleSingle)
        #threshold = ee.String(ee.Number(timeWindow['after'])).getInfo()#.get(0)
        #print(ee.String(ee.Number(threshold)).cat("asd"))
        # print((("6asd").cat((threshold))).getInfo())
        #print(ee.String("asd").cat(threshold))
        #print(ee.String("asd").cat(ee.String("asd")))
        return (meanSpace2)
  
# Making a function in order to apply the same filter over multiples ImageCollection (IC)
def filterImages(collection, startDate, endDate, region): 
  return collection.filterDate(startDate, endDate).filterBounds(region)

S1_temporal_multilooked = multitemporalDespeckle(S1, radius, units,
                                                       {'before': -6, 'after': 6, 'units': 'month'})

S1_temporal_multilooked.first()

EEException: A mapped function's arguments cannot be used in client-side operations

In [ ]:
S1_temporal_multilooked.select(ee.String(S1_temporal_multilooked.first().bandNames().get(0)).cat(".*"))

In [155]:
(ee.String(S1.first().bandNames().get(0)).getInfo())=='VV'

True

In [176]:
# def mapMeanSpace(i):
#     pol_name = ee.String(i.bandNames().get(0)) # polarization band name
#     spatial_filter_name = pol_name.cat('_multilook')
#     ## bandNamesMean_pol = pol_name.map(lambda b: ee.String(b).cat('_mean'))
#     ratio_name = pol_name.cat('_ratio')
    
#     reducer = ee.Reducer.mean()
#     kernel = ee.Kernel.square(radius, units)
#     mean = i.reduceNeighborhood(reducer, kernel).rename(spatial_filter_name) # multilook (average)
#     ratio = i.divide(mean).rename(ratio_name) # Intensity/ Imean
#     return (i.addBands(mean).addBands(ratio))
def print_fistband(i):
    print(ee.String(i.bandNames().get(0))=="VV")
#combinados = S1.select("VV").map(mapMeanSpace).combine(S1.select("VH").map(mapMeanSpace))
S1.map(print_fistband)

AttributeError: 'Image' object has no attribute 'toBands'

In [197]:


# Define a function to print the first band name of an image
def print_first_band_name(image):
    # Get the band names of the image
    #band_name = image.bandNames().get(0)
    #return(image.)
    # Print the first band name
    (print("First band name:"))

# Load an image collection
collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")#.select("B1")

# Map the function over the collection
#collection.map(print_first_band_name)
print(collection.first().bandNames().size().getInfo())

12


In [22]:
start_date="2022-01-01"
end_date="2023-05-27"
S1 = ee.ImageCollection("COPERNICUS/S1_GRD")
S1 = filterImages(S1, start_date, end_date, ROI)
radius = 3
units = 'pixels'

# S1_temporal_multilooked 12 months temporal window
S1_temporal_multilooked_12= multitemporalDespeckle(S1, radius, units,
                                                       {'before': -12, 'after': 12, 'units': 'month'})
# S1_temporal_multilooked 12 months temporal window
S1_temporal_multilooked_6= multitemporalDespeckle(S1, radius, units,
                                                       {'before': -6, 'after': 6, 'units': 'month'})
# S1_temporal_multilooked 12 months temporal window
S1_temporal_multilooked_3= multitemporalDespeckle(S1, radius, units,
                                                       {'before': -3, 'after': 3, 'units': 'month'})
# S1_temporal_multilooked 12 months temporal window
S1_temporal_multilooked_1= multitemporalDespeckle(S1, radius, units,
                                                       {'before': -1, 'after': 1, 'units': 'month'})

In [15]:
Map.addLayer(S1_temporal_multilooked_1.first(),{},"VV")
Map.addLayer(S1_temporal_multilooked_12.selec.first(),{},"Multitemporal")

In [297]:
Sites= (SM_Sites.aggregate_array('Station_na'))
output_results = "../02_Data/06_GEE_Data_extractions"

In [309]:
# Creation of a function for sampling and writing of results
def band_samplig(ImageCollection,Sites,Product_name,Individual=True):
  '''
  This function makes a sampling of each Site for each band in the ImageCollection.

  '''
  results_dir = output_results
  if not os.path.exists(results_dir): os.mkdir(results_dir)
  Sites = SM_Sites.aggregate_array('Station_na')
  Band_names= ImageCollection.first().bandNames()
  for B in range(Band_names.size().getInfo()):
    start = datetime.now()
       
    # Band selection
    Band = ImageCollection.select(Band_names.get(B).getInfo())#.toBands()
    if Individual:
      for i in range(Sites.size().getInfo()): 
        output_dir = f'{results_dir}/{Product_name}/{Band_names.get(B).getInfo()}'
        os.makedirs(output_dir,exist_ok=True)
        ID = Sites.get(i).getInfo()
        out_csv = os.path.join(output_dir, f'Site_{ID}.csv')
        print(f'Banda {Band_names.get(B).getInfo()}, Site ID: {ID}, Site {i} de {Sites.size().getInfo()}')
        # Filter Site
        Site_i = SM_Sites.filter(ee.Filter.eq("Station_na",ID))
        # Fiter S1 Bounds with the Site 1
        Band_site_i = Band.filterBounds(Site_i).toBands()
        geemap.extract_values_to_points(Site_i, Band_site_i, out_csv,verbose=False)
    else: 
      output_dir = f'{results_dir}/{Product_name}/{Band_names.get(B).getInfo()}'
      os.makedirs(output_dir,exist_ok=True)
      out_csv = os.path.join(output_dir, f'Sites_{Band_names.get(B).getInfo()}.csv')
      Band = Band.toBands()
      #Band_B = ee.ImageCollection(Band)
      #geemap.extract_values_to_points(Sites, Band, out_csv,verbose=False)
      #Site_i = SM_Sites.filter(ee.Filter.eq("Station_na",'B1'))
      geemap.extract_values_to_points(SM_Sites, Band, out_fc=out_csv, scale=30, \
        crs=None, crsTransform=None, tileScale=1, stats_type='FIRST', \
        timeout=1200, proxies=None)
    print('Duration: {}'.format(datetime.now()-start))

In [606]:
#,'VV_multilook','VH_multilook','VV_temporalfilter','VH_temporalfilter'
band_samplig(S1_temporal_multilooked_1.select(['VV','VH','VV_multilook','VH_multilook']),\
    SM_Sites,'Sentinel_1',False)
band_samplig(S1_temporal_multilooked_12.select(['.*temporalfilter']),\
    SM_Sites,'Sentinel_1',False)
band_samplig(S1_temporal_multilooked_6.select(['.*temporalfilter']),\
    SM_Sites,'Sentinel_1',False)
band_samplig(S1_temporal_multilooked_3.select(['.*temporalfilter']),\
    SM_Sites,'Sentinel_1',False)
band_samplig(S1_temporal_multilooked_1.select(['.*temporalfilter']),\
    SM_Sites,'Sentinel_1',False)

Generating URL ...
Please wait ...
Data downloaded to d:\Documents\Fran\MAIE\14_Tutoria_de_investigacion\A_Proyects\Italy_Greifeneder_Model_Validation\02_Data\06_GEE_Data_extractions\Sentinel_1\VV_12_month_temporalfilter\Sites_VV_12_month_temporalfilter.csv
Duration: 0:00:34.291714
Generating URL ...
Please wait ...
Data downloaded to d:\Documents\Fran\MAIE\14_Tutoria_de_investigacion\A_Proyects\Italy_Greifeneder_Model_Validation\02_Data\06_GEE_Data_extractions\Sentinel_1\VH_12_month_temporalfilter\Sites_VH_12_month_temporalfilter.csv
Duration: 0:00:18.245068
Generating URL ...
Please wait ...
Data downloaded to d:\Documents\Fran\MAIE\14_Tutoria_de_investigacion\A_Proyects\Italy_Greifeneder_Model_Validation\02_Data\06_GEE_Data_extractions\Sentinel_1\VV_6_month_temporalfilter\Sites_VV_6_month_temporalfilter.csv
Duration: 0:00:15.954908
Generating URL ...
Please wait ...
Data downloaded to d:\Documents\Fran\MAIE\14_Tutoria_de_investigacion\A_Proyects\Italy_Greifeneder_Model_Validation\02